In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import poisson
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
def sqrt_func(i, j):
    time.sleep(1)
    return i + j


Parallel(n_jobs=2)(delayed(sqrt_func)(i, 5)
                   for i in range(5))


[5, 6, 7, 8, 9]

## Functions

In [3]:
I = 48  # Number of intervals
d = 5  # length of interval
x = np.random.choice(5, I)
N = sum(x)
precision = 0.9999
beta = 9  # Average service time for a patient
no_show = 0 # Fraction of scheduled patients not showing up
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

print(N)

91


In [4]:
def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))

In [5]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))


def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [6]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

##########
## TEST ##
##########

probS = calcExponentialLimit(beta*N)+1
print(probS)
calculate_p(beta, probS, precision)


934


([0.00012340980408667956,
  0.0011106882367801166,
  0.004998097065510523,
  0.014994291196531574,
  0.033737155192196056,
  0.06072687934595293,
  0.09109031901892926,
  0.1171161244529091,
  0.13175564000952278,
  0.13175564000952278,
  0.11858007600857066,
  0.09702006218883041,
  0.0727650466416229,
  0.050375801521123684,
  0.03238444383500792,
  0.01943066630100472,
  0.010929749794315179,
  0.005786338126402172,
  0.0028931690632010803,
  0.0013704485036215655,
  0.0006167018266297063,
  0.0002643007828413016,
  0.00010812304752598687,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [7]:
def calculate_v(p, beta, precision, precision_limit, N, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*N) + 1
  v = np.zeros((N+1, limit+d))
  add_v = np.zeros((N+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, N+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i]
      i += 1

  for k in range(N+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  return v, limit

##########
## TEST ##
##########

p, precision_limit = calculate_p(beta, probS, precision)
calculate_v(p, beta, precision, precision_limit, N, d, no_show)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.10164981e-08, 2.95789342e-08, 1.35645267e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 933)

In [8]:
def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1, limit+d+1))
  p_min = np.zeros((I+1, limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1):  # calculate p_min and p_plus iteratively
    # Constraint 3
    for k in range(d+1):
      # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
      p_min[t][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1, limit+1):

      # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:  # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

##########
## TEST ##
##########

v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)
p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)


In [9]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k]  # I+1
  return tardiness


def calcIdletime(I, d, tardiness, N, no_show, beta):
  return (I * d) + tardiness - (N * (1 - no_show) * beta)  # I-1?


def calcWaitingtime(p_min, x, p, limit, I, N):
  w = np.zeros((I+1, N+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k]
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= N
  return waitingtime

##########
## TEST ##
##########

calcWaitingtime(p_min, x, p, limit, I, N)


288.0252797016496

In [10]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess


##########
## TEST ##
##########

# calcFracExcess(p_min, I)


<div>
<img src="https://github.com/witusj/obp/blob/master/images/fitness2.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/fitness.png?raw=true" width="450">
</div>

In [11]:
def calcFitness(x, beta, precision, limit, v, N, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, N, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, N)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  print(f"Schedule: {x},\nN: {sum(x)},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

  # Collect into a dictionary
  results = {'p_min': p_min, 'waitingTime': waitingtime,
            'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess

  return results

##########
## TEST ##
##########


p, precision_limit = calculate_p(beta, probS, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)



results = calcFitness(x, beta, precision, limit, v, N, no_show,
                  I, d, eind, alpha_W, alpha_I, alpha_T)
results


Schedule: [0 2 1 4 0 0 0 4 4 3 0 2 3 1 3 1 4 1 2 4 1 4 0 0 4 3 4 3 0 2 0 2 0 2 1 0 3
 0 0 2 3 4 4 2 1 0 3 4],
N: 91,
Objective value: 347.6328937685963,
Prob calculation time: 16.050288 sec,
Waiting time (timer): 288.0252797016496 (33.920290 sec),
Idle time (timer): 1.3713031465607628 (0.000002 sec),
Tardiness (timer): 580.3713031465608 (0.000314 sec)



{'p_min': array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [1.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [3.23993451e-004, 7.19452059e-004, 1.85001958e-003, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        ...,
        [3.06054170e-117, 1.08941586e-116, 4.78093617e-116, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [1.89674171e-123, 9.57428251e-123, 5.05879280e-122, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [5.79567821e-133, 3.92874979e-132, 2.64161073e-131, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000]]),
 'waitingTime': 288.0252797016496,
 'idleTime': 1.3713031465607628,
 'tardiness': 580.3713031465608,
 'objVal': 347.6328937685963}

In [12]:
def createDist(arr, reverse=False):
    minX, maxX = arr.min(), arr.max()
    minY, maxY = 0, 1
    if(reverse):
        minY, maxY = maxY, minY
    # Scale to values between 0, 1 and make total sum equal to 1
    arrScaled = np.interp(arr, (minX, maxX), (minY, maxY))
    arrNorm = [x / arrScaled.sum() for x in arrScaled]
    return(arrNorm)
    

##########
## TEST ##
##########

arr = np.random.choice(10, 5)
dist1 = createDist(arr)
dist2 = createDist(arr, reverse=True)
arr, dist1, sum(dist1), dist2, sum(dist2)


(array([8, 0, 5, 1, 7]),
 [0.38095238095238093,
  0.0,
  0.23809523809523808,
  0.047619047619047616,
  0.3333333333333333],
 1.0,
 [0.0,
  0.42105263157894735,
  0.15789473684210525,
  0.3684210526315789,
  0.05263157894736842],
 1.0)

<div>
<img src="https://github.com/witusj/obp/blob/master/images/crossover.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/crossover2.png?raw=true" width="450">
</div>

In [13]:
# OLD
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    p1 = createDist(child1)
    
    # Swap patients parent1 to parent 2
    n1 = nSwaps
    while(n1 > 0):
        i = np.random.choice(I, 1, p=p1)
        child1[i] += -1
        child2[i] += 1
        if(child1[i] == 0):
            p1 = createDist(child1) # If the number of patients in interval i reaches zero, this interval should not be selected again.
        n1 -= 1
    
    # Swap patients parent2 to parent 1
    p2 = createDist(child2)
    
    n2 = nSwaps
    while(n2 > 0):
        i = np.random.choice(I, 1, p=p2)
        child2[i] += -1
        child1[i] += 1
        if(child2[i] == 0):
            p2 = createDist(child2)
        n2 -= 1
        
    return(child1, child2)

##########
## TEST ##
##########

testI = 10
parent1 = np.random.choice(5, testI)
parent2 = np.random.choice(5, testI)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[4 0 1 3 2 0 0 2 3 3] [4 3 4 2 3 4 3 2 0 0]


(array([4, 0, 2, 2, 2, 0, 1, 2, 2, 3]), array([4, 3, 3, 3, 3, 4, 2, 2, 1, 0]))

In [14]:
# NEW
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    p1 = createDist(child1)
    p2 = createDist(child2, reverse=True)
    
    # Swap patients parent1 to parent 2
    n1 = nSwaps
    while(n1 > 0):
        i = np.random.choice(I, 1, p=p1)
        j = np.random.choice(I, 1, p=p2)
        child1[i] += -1
        child2[j] += 1
        if(child1[i] == 0):
            p1 = createDist(child1) # If the number of patients in interval i reaches zero, this interval should not be selected again.
        n1 -= 1
    
    # Swap patients parent2 to parent 1
    p2 = createDist(child2)
    p1 = createDist(child1, reverse=True)
    
    n2 = nSwaps
    while(n2 > 0):
        i = np.random.choice(I, 1, p=p2)
        j = np.random.choice(I, 1, p=p1)
        child2[i] += -1
        child1[j] += 1
        if(child2[i] == 0):
            p2 = createDist(child2)
        n2 -= 1
        
    return(child1, child2)

##########
## TEST ##
##########

testI = 10
parent1 = np.random.choice(5, testI)
parent2 = np.random.choice(5, testI)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[2 1 3 2 2 4 3 4 4 3] [1 2 1 4 3 0 1 4 1 4]


(array([1, 2, 2, 3, 2, 4, 3, 3, 4, 4]), array([1, 1, 1, 3, 3, 1, 3, 4, 1, 3]))


<div>
    <img src="https://github.com/witusj/obp/blob/master/images/mutation.png?raw=true" width="450">
    <img src="https://github.com/witusj/obp/blob/master/images/mutation1.png?raw=true" width="450">
</div>

In [15]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[1 2 2 3 2 4 3 3 4 4]
[1 2 3 3 2 4 2 3 4 4]


## Classes

#### Suggesties: [get / set methods](https://www.geeksforgeeks.org/getter-and-setter-in-python/) voor parameters / met tests / nuttig voor optimaliseren hyper parameters, bv mbv bayesian stats

Create a data object to store and generate initial data.

In [16]:
class Data:
    def __init__(self, I=10, N=12, d=5, popSize=5, beta=9, precision=0.9999, no_show=0, eind=0, alpha_I=0.2, alpha_T=0.3, alpha_W=0.5, nSwaps=4, mutationRate=0.1):
        self.I = I
        self.N = N
        self.d = d
        self.popSize = popSize
        self.beta = beta
        self.precision = precision
        self.no_show = no_show
        self.probSize = calcExponentialLimit(self.beta*self.N)+1
        self.p, self.precision_limit = self.__calculate_p()
        self.v, self.limit = self.__calculate_v()
        self.eind = eind
        self.alpha_I, self.alpha_T, self.alpha_W = alpha_I, alpha_T, alpha_W
        self.nSwaps, self.mutationRate = nSwaps, mutationRate
    
    def get_initial_data(self): return np.array((self.I, self.N, self.d, self.beta, self.popSize, self.eind, self.alpha_I, self.alpha_T, self.alpha_W, self.nSwaps, self.mutationRate),
                                                dtype={'names': ('I', 'N', 'd', 'beta', 'popSize', 'eind', 'alpha_I', 'alpha_T', 'alpha_W', 'nSwaps', 'mutationRate'),
                                                'formats': ('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'f8', 'f8', 'f8', 'i4', 'f8')})

    def get_prob_data(self): return np.array((self.precision, self.no_show, self.probSize, self.precision_limit, self.limit),
                                                dtype={'names': ('precision', 'no_show', 'probSize', 'precision_limit', 'limit'),
                                                'formats': ('f8', 'f8', 'i4', 'f8', 'i4')})

    def calculate_p(self):  return calculate_p(self.beta, self.probSize, self.precision)
    __calculate_p = calculate_p
    
    def calculate_v(self): return calculate_v(self.p, self.beta, self.precision, self.precision_limit, self.N, self.d, self.no_show)
    __calculate_v = calculate_v
    

##########
## TEST ##
##########

newDataObject = Data(I, N)
initData = newDataObject.get_initial_data()
initData[['N', 'popSize']]


array((91, 5),
      dtype={'names': ['N', 'popSize'], 'formats': ['<i4', '<i4'], 'offsets': [4, 16], 'itemsize': 60})

<img src="https://github.com/witusj/obp/blob/master/images/population.png?raw=true" width="600">

Create a random individual. An individual is in this case one schedule.

In [17]:
class Schedule:
    def __init__(self, dataObject):
        self.I = dataObject.get_initial_data()['I']  # Number of intervals
        self.N = dataObject.get_initial_data()['N']  # Number of patients
        self.x = np.zeros(self.I).astype(int)# Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1

    def create_bw_schedule(self):
        self.x = np.ones(self.I).astype(int)
        self.x[0] = 2
        checkSum = np.sum(self.x)
        print(f'checkSum = {checkSum}')
        diff = checkSum - self.N
        print(f'diff = {diff}')
        if diff > 0:
            i = np.random.choice(range(1, self.I), diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] - 1
        if diff < 0:
            i = np.random.choice(range(1, self.I), -diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)


##########
## TEST ##
##########

dataObject = Data(I=I, N=N, d=d)
scheduleObject = Schedule(dataObject)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
[1 2 4 1 3 3 2 5 3 2 1 1 0 2 3 2 0 1 1 1 1 1 6 0 3 2 3 2 4 3 4 2 1 2 1 2 1
 0 0 1 3 1 2 1 1 1 3 2]
91
checkSum = 49
diff = -42
i = [15 33 41 26 44 11 18 24 38 14 16  1  8 45  7 28 19 10 27 32 47 39 31 23
 36 17  5 35 46 43 25 21 13 42 29 37  2 30  9 40 12 20]
[2 2 2 1 1 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 1 2 2
 2 2 2 2 2 2 2 2 2 2 2]
91


Create population of individuals

In [18]:
# OLD
class Population:
    global bestRank
    bestRank = []
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in self.individuals])
            
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = self.params['nSwaps']*1 ## Times 1 because otherwise refers to memorypoint
        mutationRate = self.params['mutationRate']
        self.population_dist = createDist(self.fitnessScores, reverse=True)
        
        for i in range(self.params['popSize']):
            print(f'Crossover {i}')
            r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
            parent1, parent2 = self.individuals[r1], self.individuals[r2]
            child1, child2 = crossover(parent1, parent2, nSwaps)
            newIndividuals.append(mutate(child1, mutationRate))
            newIndividuals.append(mutate(child2, mutationRate))
            
        beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
            'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
        precision, limit, no_show = self.prob_params[[
            'precision', 'limit', 'no_show']].tolist()
        self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show,
                                            I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in newIndividuals])
        tempList = list(range(self.params['popSize']*2))
        zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
        self.fitnessScores = np.array(list(zipped[0][:self.params['popSize']]))
        tempList = zipped[1][:self.params['popSize']]
        self.individuals = list(map(newIndividuals.__getitem__, tempList))
        bestRank.append([self.fitnessScores[0], self.individuals[0]])
        print(self.fitnessScores, '\n', self.individuals)


## Tests

In [19]:
##########
## TEST ##
##########

bestRank = []
dataObject = Data()
populationObject1 = Population(dataObject)
populationObject1.fitnessScores
populationObject1.calc_fitness_scores()
print(populationObject1.fitnessScores)


Schedule: [5 1 1 2 1 0 1 0 1 0],
N: 12,
Objective value: 36.483989758339234,
Prob calculation time: 0.087648 sec,
Waiting time (timer): 38.226143439432526 (0.073751 sec),
Idle time (timer): -0.058163922754062014 (0.000002 sec),
Tardiness (timer): 57.94183607724594 (0.000052 sec)

Schedule: [3 2 2 0 0 1 3 0 1 0],
N: 12,
Objective value: 34.4016030264544,
Prob calculation time: 0.083701 sec,
Waiting time (timer): 34.06136988552605 (0.101995 sec),
Idle time (timer): -0.058163832617239564 (0.000002 sec),
Tardiness (timer): 57.94183616738276 (0.000052 sec)

Schedule: [1 2 0 2 1 2 0 2 1 1],
N: 12,
Objective value: 31.154111785416116,
Prob calculation time: 0.085193 sec,
Waiting time (timer): 27.477752861575038 (0.090724 sec),
Idle time (timer): 0.030470709257201634 (0.000002 sec),
Tardiness (timer): 58.030470709257195 (0.000052 sec)

Schedule: [1 1 1 0 0 3 2 1 2 1],
N: 12,
Objective value: 30.265027908242416,
Prob calculation time: 0.084992 sec,
Waiting time (timer): 24.574307462794224 (0.07

In [20]:
individuals2 = populationObject1.individuals.copy()
populationObject2 = Population(dataObject, individuals=individuals2)
print(populationObject1.individuals, '\n', populationObject2.individuals)

[array([5, 1, 1, 2, 1, 0, 1, 0, 1, 0]), array([3, 2, 2, 0, 0, 1, 3, 0, 1, 0]), array([1, 2, 0, 2, 1, 2, 0, 2, 1, 1]), array([1, 1, 1, 0, 0, 3, 2, 1, 2, 1]), array([1, 2, 1, 1, 1, 0, 3, 0, 1, 2])] 
 [array([5, 1, 1, 2, 1, 0, 1, 0, 1, 0]), array([3, 2, 2, 0, 0, 1, 3, 0, 1, 0]), array([1, 2, 0, 2, 1, 2, 0, 2, 1, 1]), array([1, 1, 1, 0, 0, 3, 2, 1, 2, 1]), array([1, 2, 1, 1, 1, 0, 3, 0, 1, 2])]


In [21]:
populationObject2.calc_fitness_scores()
populationObject2.apply_crossover()
populationObject2.individuals


Schedule: [5 1 1 2 1 0 1 0 1 0],
N: 12,
Objective value: 36.483989758339234,
Prob calculation time: 0.093268 sec,
Waiting time (timer): 38.226143439432526 (0.088231 sec),
Idle time (timer): -0.058163922754062014 (0.000002 sec),
Tardiness (timer): 57.94183607724594 (0.000063 sec)

Schedule: [3 2 2 0 0 1 3 0 1 0],
N: 12,
Objective value: 34.4016030264544,
Prob calculation time: 0.085839 sec,
Waiting time (timer): 34.06136988552605 (0.108698 sec),
Idle time (timer): -0.058163832617239564 (0.000002 sec),
Tardiness (timer): 57.94183616738276 (0.000054 sec)

Schedule: [1 2 0 2 1 2 0 2 1 1],
N: 12,
Objective value: 31.154111785416116,
Prob calculation time: 0.090251 sec,
Waiting time (timer): 27.477752861575038 (0.085535 sec),
Idle time (timer): 0.030470709257201634 (0.000003 sec),
Tardiness (timer): 58.030470709257195 (0.000057 sec)

Schedule: [1 1 1 0 0 3 2 1 2 1],
N: 12,
Objective value: 30.265027908242416,
Prob calculation time: 0.078786 sec,
Waiting time (timer): 24.574307462794224 (0.07

[array([1, 2, 0, 2, 0, 1, 2, 1, 2, 1]),
 array([1, 1, 1, 2, 1, 1, 2, 0, 1, 2]),
 array([1, 0, 2, 1, 1, 1, 3, 0, 1, 2]),
 array([3, 1, 1, 1, 0, 1, 1, 2, 1, 1]),
 array([1, 2, 1, 2, 0, 2, 1, 2, 1, 0])]

In [22]:
# Initial population
bestRank = []
dataObject = Data()
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
individuals = populationZero.individuals.copy()
generations = 3

# Genetic algorithm loop
for i in range(generations):
    print(f'Generation: {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.calc_fitness_scores()
    populationObject.apply_crossover()
    individuals = populationObject.individuals


Schedule: [1 2 0 1 0 1 1 3 3 0],
N: 12,
Objective value: 29.52549125994123,
Prob calculation time: 0.087087 sec,
Waiting time (timer): 24.17533825826074 (0.083576 sec),
Idle time (timer): 0.07564426162171856 (0.000002 sec),
Tardiness (timer): 58.07564426162172 (0.000053 sec)

Schedule: [1 1 2 1 1 1 1 1 2 1],
N: 12,
Objective value: 30.74563560743352,
Prob calculation time: 0.082545 sec,
Waiting time (timer): 26.65231750930347 (0.041543 sec),
Idle time (timer): 0.03895370556357136 (0.000003 sec),
Tardiness (timer): 58.03895370556357 (0.000052 sec)

Schedule: [3 1 1 0 0 1 0 2 2 2],
N: 12,
Objective value: 30.449691749716056,
Prob calculation time: 0.079420 sec,
Waiting time (timer): 26.151595385310586 (0.104412 sec),
Idle time (timer): -0.052211885878477915 (0.000002 sec),
Tardiness (timer): 57.94778811412153 (0.000051 sec)

Schedule: [6 1 1 0 0 1 1 0 1 1],
N: 12,
Objective value: 35.65131024456185,
Prob calculation time: 0.082305 sec,
Waiting time (timer): 36.56078485773253 (0.062829 se

In [23]:

bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF


,Score,Schedule
0,29.324711,"[1, 1, 2, 0, 0, 1, 1, 3, 2, 1]"
1,29.085297,"[1, 1, 1, 1, 1, 1, 0, 2, 2, 2]"
2,28.375547,"[1, 1, 1, 0, 1, 1, 2, 0, 2, 3]"


In [24]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 0 0 0 0 2 0 0 0 2 1 1 1 1 0 0 1 1 0 0 2 1 0 2 1 1 0 0 1 0 0 1 0 0 1 1 0
 0 0 1 1 0 0 0 0 1 0 0],
N: 24,
Objective value: 19.05518271620216,
Prob calculation time: 1.472527 sec,
Waiting time (timer): 23.957140880544813 (0.277222 sec),
Idle time (timer): 31.7872106066404 (0.000002 sec),
Tardiness (timer): 7.7872106066403886 (0.000093 sec)

Schedule: [1 1 1 1 0 2 1 1 0 0 1 0 1 0 0 0 2 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 0
 0 1 1 0 1 0 0 0 0 0 2],
N: 24,
Objective value: 16.546757741405,
Prob calculation time: 1.440989 sec,
Waiting time (timer): 9.762984141072236 (0.227354 sec),
Idle time (timer): 37.06927347496017 (0.000003 sec),
Tardiness (timer): 13.069273474960173 (0.000100 sec)

Schedule: [1 0 0 0 0 1 1 1 2 0 2 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 2 1 1 0 0 0
 0 0 0 0 0 0 1 0 1 2 0],
N: 24,
Objective value: 19.310960393434996,
Prob calculation time: 1.426104 sec,
Waiting time (timer): 16.52563148302134 (0.286404 sec),
Idle time (timer): 37.16784633371077 (0.000002 se

In [25]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='LimeGreen')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [26]:
scheduleObject = Schedule(dataObject)
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
x = scheduleObject.get_schedule()
calcFitness(x, beta, precision, limit, v, N, no_show,
            I, d, eind, alpha_W, alpha_I, alpha_T)


checkSum = 49
diff = 25
i = [14 45 18 13 23 15 36  5  6  8  1 32 35 10 44 22 16 34 42 19 31 26 39  4
 20]
[2 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0
 1 1 0 1 1 0 1 0 0 1 1]
24
Schedule: [2 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0
 1 1 0 1 1 0 1 0 0 1 1],
N: 24,
Objective value: 14.685032554549426,
Prob calculation time: 16.429987 sec,
Waiting time (timer): 5.989664649960581 (0.879080 sec),
Idle time (timer): 36.481944490942 (0.000002 sec),
Tardiness (timer): 12.481944490941984 (0.000319 sec)



{'p_min': array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [3.23993451e-04, 7.19452059e-04, 1.85001958e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [3.03662559e-02, 2.45211686e-02, 3.67817529e-02, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [4.07803333e-01, 5.60113207e-02, 5.51027406e-02, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [5.18277959e-02, 4.35181150e-02, 5.90582297e-02, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.01914806e-02, 1.09113145e-02, 1.76807710e-02, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 'waitingTime': 5.989664649960581,
 'idleTime': 36.481944490942,
 'tardiness': 12.481944490941984,
 'objVal': 14.685032554549426}

In [27]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 10

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 2 2 0 0 1 0 1 1 0 0 0 0 2 1 1 0 2 1 0
 0 1 0 1 0 2 0 0 0 1 1],
N: 24,
Objective value: 31.30485915077309,
Prob calculation time: 1.516217 sec,
Waiting time (timer): 17.669796527482468 (0.383674 sec),
Idle time (timer): 56.39490089963351 (0.000002 sec),
Tardiness (timer): 32.39490089963349 (0.000117 sec)

Schedule: [0 0 0 0 0 1 1 0 1 2 0 0 2 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 2 1 0 0 0 0
 0 1 0 0 1 0 0 1 2 1 2],
N: 24,
Objective value: 30.367658495885152,
Prob calculation time: 1.567421 sec,
Waiting time (timer): 11.624655367598693 (0.388828 sec),
Idle time (timer): 58.86299391474279 (0.000002 sec),
Tardiness (timer): 34.86299391474279 (0.000114 sec)

Schedule: [0 1 2 0 0 3 0 2 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0
 0 0 0 1 0 0 3 1 0 1 2],
N: 24,
Objective value: 34.09623719763999,
Prob calculation time: 1.604940 sec,
Waiting time (timer): 16.27901077022916 (0.460148 sec),
Idle time (timer): 61.974388149247204 (0.000002

In [28]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='RoyalBlue')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [29]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 20

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [0 0 1 0 0 0 0 0 3 1 0 0 1 0 2 1 1 0 1 0 0 0 1 0 2 1 0 2 0 1 0 0 0 1 1 0 1
 0 0 0 0 1 1 0 0 1 0 0],
N: 24,
Objective value: 19.81754870884412,
Prob calculation time: 1.434932 sec,
Waiting time (timer): 22.733889377092854 (0.324295 sec),
Idle time (timer): 33.873321596678295 (0.000002 sec),
Tardiness (timer): 9.873321596678304 (0.000093 sec)

Schedule: [1 1 2 0 0 1 0 1 0 0 1 0 1 0 1 0 0 1 0 0 2 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 1
 1 1 1 0 2 0 0 0 0 0 0],
N: 24,
Objective value: 9.803512628805704,
Prob calculation time: 1.422592 sec,
Waiting time (timer): 7.947761228788802 (0.218315 sec),
Idle time (timer): 27.040680228816967 (0.000002 sec),
Tardiness (timer): 3.040680228816975 (0.000099 sec)

Schedule: [0 0 0 1 1 0 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 2 1 1 0 0 0 1 0 0 0 0 0 2 2 0 1
 0 1 1 0 0 0 2 0 0 0 1],
N: 24,
Objective value: 20.30504300831086,
Prob calculation time: 1.413307 sec,
Waiting time (timer): 12.574319181320186 (0.296454 sec),
Idle time (timer): 41.45885889297131 (0.000002

In [30]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='Tomato')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [31]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [1 0 1 0 0 1 0 1 0 2 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 2 3 0 1 0 0 1
 0 0 0 0 1 0 2 2 0 1 0],
N: 24,
Objective value: 26.7692903934408,
Prob calculation time: 1.370942 sec,
Waiting time (timer): 11.367175193294573 (0.374293 sec),
Idle time (timer): 53.03736719353827 (0.000002 sec),
Tardiness (timer): 29.03736719353829 (0.000093 sec)

Schedule: [0 0 1 1 1 0 0 0 0 1 2 1 0 0 1 2 1 0 0 0 0 0 1 1 1 1 0 0 1 0 2 1 1 0 1 0 0
 0 1 1 1 0 0 0 0 0 0 0],
N: 24,
Objective value: 13.133998148057898,
Prob calculation time: 1.406729 sec,
Waiting time (timer): 16.10130382615611 (0.228267 sec),
Idle time (timer): 27.155794362659094 (0.000002 sec),
Tardiness (timer): 3.155794362659088 (0.000094 sec)

Schedule: [1 0 1 0 1 0 0 0 0 1 0 0 3 0 0 1 0 0 0 1 0 1 1 0 1 0 1 2 0 2 0 0 2 0 0 0 0
 0 0 0 1 1 1 2 0 0 0 0],
N: 24,
Objective value: 14.293659222921722,
Prob calculation time: 1.432673 sec,
Waiting time (timer): 10.450426594961579 (0.399969 sec),
Idle time (timer): 32.855814308228474 (0.00000

In [32]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkOrange')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [33]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [2 1 0 0 0 2 0 0 1 0 1 0 0 0 0 0 0 1 2 2 1 0 0 0 0 2 0 1 1 0 0 2 0 0 1 0 0
 0 1 0 0 1 0 1 0 1 0 0],
N: 24,
Objective value: 12.817600486770381,
Prob calculation time: 1.541181 sec,
Waiting time (timer): 13.917172766290335 (0.541813 sec),
Idle time (timer): 28.084552300423752 (0.000002 sec),
Tardiness (timer): 4.084552300423742 (0.000116 sec)

Schedule: [0 0 1 1 0 0 1 2 2 0 0 2 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0
 1 0 1 2 1 1 0 1 1 0 0],
N: 24,
Objective value: 20.875559855493236,
Prob calculation time: 1.906811 sec,
Waiting time (timer): 11.902892024536008 (0.276471 sec),
Idle time (timer): 42.85733840946472 (0.000002 sec),
Tardiness (timer): 18.857338409464717 (0.000096 sec)

Schedule: [1 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 2 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0
 3 2 2 0 1 0 0 0 0 1 0],
N: 24,
Objective value: 26.03130167770566,
Prob calculation time: 1.511644 sec,
Waiting time (timer): 13.753783405585986 (0.321235 sec),
Idle time (timer): 50.21631385911877 (0.000

In [34]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkCyan')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")
